# Initialization

## Imports

In [2]:
from pathlib import Path
from subprocess import call

import pandas as pd
import yaml

## Useful functions

In [11]:
_FASTQ_DUMP_BIN = '/Users/dstone/software/sratoolkit.2.9.6-1-mac64/bin/fastq-dump' # dependent on your SRA installation
def download_sra_fasta(sra_num: str, download_path: str=None, overwrite_existing: bool=False) -> int:
    out_file = Path(download_path or './', f'{sra_num}.fasta').resolve()
    if not overwrite_existing and out_file.exists():
        print(f'{sra_num} already exists. Passing.')
        return 0
    
    call_cmd = [_FASTQ_DUMP_BIN, '--fasta', sra_num]
    if download_path is not None:
          call_cmd.extend(['--outdir', str(Path(download_path).resolve())])
    print(f'Downloading {sra_num} to fasta...')
    ret_value = call(call_cmd)
    if ret_value != 0:
        raise Exception(f'Received return value from fastq-dump: {ret_value}')
    print(f'Done!')
    return ret_value

_GENBANK_URL = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id={acn}&rettype=fasta'
def download_genbank_fasta(accession_num: str, download_path: str=None, overwrite_existing: bool=False) -> int:
    out_file = Path(download_path or './', f'{accession_num}.fasta').resolve()
    if not overwrite_existing and out_file.exists():
        print(f'{accession_num} already exists. Passing.')
        return 0
    
    call_cmd = ['wget', _GENBANK_URL.format(acn=accession_num)]
    if download_path is not None:
        call_cmd.extend(['--output-document', str(Path(download_path, f'{accession_num}.fasta').resolve())])
    else:
        call_cmd.extend(['--output-document', f'{accession_num}.fasta'])
    print(f'Downloading {accession_num} to fasta...')
    ret_value = call(call_cmd)
    if ret_value != 0:
        raise Exception(f'Received return value from wget: {ret_value}')
    print(f'Done!')
    return ret_value

## Download latest ncov sequence accession/SRA numbers

NCBI is maintaining a `.yaml` file that has Genbank accession + SRA run numbers, along with metadata, at the following url

In [4]:
ncov_yaml_path = '/Users/dstone/projects/covid-19/ncov-sequences.yaml'
!wget https://www.ncbi.nlm.nih.gov/core/assets/genbank/files/ncov-sequences.yaml \
    --output-document  $ncov_yaml_path

--2020-04-12 15:48:48--  https://www.ncbi.nlm.nih.gov/core/assets/genbank/files/ncov-sequences.yaml
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95960 (94K)
Saving to: ‘/Users/dstone/projects/covid-19/ncov-sequences.yaml’

/Users/dstone/proje 100%[===================>]  93.71K   389KB/s    in 0.2s    

2020-04-12 15:48:49 (389 KB/s) - ‘/Users/dstone/projects/covid-19/ncov-sequences.yaml’ saved [95960/95960]



In [5]:
# now open that file
with open(ncov_yaml_path, 'r') as stream:
    ncov_data = yaml.safe_load(stream)

In [7]:
ncov_data['genbank-sequences'][0]

{'accession': 'MN908947',
 'refseq-accession': 'NC_045512',
 'gene-region': 'complete',
 'collection-date': '2019-12',
 'locality': {'country': 'China'}}

In [12]:
# acquire all genbank data
for genbank_dict in ncov_data['genbank-sequences']:
    acn, completeness = genbank_dict['accession'], genbank_dict['gene-region']
    if completeness == 'complete':
        # only download if we have a full genome
        download_genbank_fasta(acn, download_path='/Users/dstone/projects/covid-19/fastas/genbank/')

MN908947 already exists. Passing.
MN938384 already exists. Passing.
MN975262 already exists. Passing.
MN985325 already exists. Passing.
MN988713 already exists. Passing.
MN994467 already exists. Passing.
MN994468 already exists. Passing.
MN997409 already exists. Passing.
MN988668 already exists. Passing.
MN988669 already exists. Passing.
MN996527 already exists. Passing.
MN996528 already exists. Passing.
MN996529 already exists. Passing.
MN996530 already exists. Passing.
MN996531 already exists. Passing.
MT007544 already exists. Passing.
LR757995 already exists. Passing.
LR757996 already exists. Passing.
LR757998 already exists. Passing.
MT019529 already exists. Passing.
MT019530 already exists. Passing.
MT019531 already exists. Passing.
MT019532 already exists. Passing.
MT019533 already exists. Passing.
MT020880 already exists. Passing.
MT020881 already exists. Passing.
MT027062 already exists. Passing.
MT027063 already exists. Passing.
MT027064 already exists. Passing.
MT039873 alrea

In [1]:
# this stalls after the first few downloads every time, so will stick with the genbank sequences for now
# # acquire all SRA data
# for genbank_dict in ncov_data['sra-accessions']:
#     sra_num = genbank_dict['sra-run']
#     download_sra_fasta(sra_num, download_path='/Users/dstone/projects/covid-19/fastas/sra/', overwrite_existing=False)

## Create dataframe of all accession information

In [69]:
ncov_data['genbank-sequences'][0]

{'refseq-accession': 'NC_045512',
 'gene-region': 'complete',
 'collection-date': '2019-12',
 'locality': {'country': 'China'}}

In [81]:
metadata_fields = ['collection-date', 'country', 'state']
df_metadata = pd.DataFrame(columns=metadata_fields)
for sd in ncov_data['genbank-sequences']:
    row_dict = {}
    for mf in metadata_fields:
        if mf in ['country', 'state']:
            row_dict[mf] = sd['locality'].get(mf)
        else:
            row_dict[mf] = sd[mf]
    df_metadata.loc[sd['accession']] = row_dict

# Running analysis on sequences

### Run MUSCLE locally

Get MUSCLE:
```bash
 wget http://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86darwin64.tar.gz --directory-prefix ~/projects/covid-19/
 cd ~/projects/covid-19/
 tar -xvzf muscle3.8.31_i86darwin64.tar.gz # yields a binary
 # combine above download fastas into one fasta for alignment
 cd ~/projects/covid-19/fastas/genbank
 # assumes there is one read per fasta (the above download code does yield this)
 cat *.fasta > all_ncov_genomes.fa
 # run MUSCLE, use default parameters
 cd ~/projects/covid-19/
 ./muscle3.8.31_i86darwin64 -in fastas/genbank/all_ncov_genomes.fa -out ./ncov_muscle_alignments.fa
 # time taken: ~
```

Took too long (at least 24 hours). Current output:
```
dstone@5prime:~/projects/covid-19$ ./muscle3.8.31_i86darwin64 -in fastas/genbank/all_ncov_genomes.fa -out ./ncov_muscle_alignments.fa

MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

all_ncov_genomes 485 seqs, max length 29945, avg  length 29847
00:00:22     79 MB(0%)  Iter   1  100.00%  K-mer dist pass 1
00:00:22     79 MB(0%)  Iter   1  100.00%  K-mer dist pass 2
02:25:18  4706 MB(27%)  Iter   1  100.00%  Align node       
02:25:18  4727 MB(28%)  Iter   1  100.00%  Root alignment
04:26:24  4727 MB(28%)  Iter   2  100.00%  Refine tree   
04:26:24  4727 MB(28%)  Iter   2  100.00%  Root alignment
04:26:24  4727 MB(28%)  Iter   2  100.00%  Root alignment
08:48:38  4727 MB(28%)  Iter   3  100.00%  Refine biparts
^[[B^[[B  4727 MB(28%)  Iter   4   40.33%  Refine biparts
```

**Solution: look up if it is memory or CPU intensive and spin up an AWS ma

### Build phylogenetic tree from MUSCLE output

## Alternate Method of building tree: k-mer based

### Run jobs to compute k-mer profiles for each strain downloaded

### For varying values of `k`, use D_2 metric to compute distances

### Build phylogenetic tree from this distance method

### Compare with results from MUSCLE-alignment tree